In [1]:
import numpy as np
#np.random.seed(50)

In [2]:
class mul_lay_prcp_layer():
    def __init__(self, no_of_prev_lay_node, no_of_curr_lay_node, last_lay = False, bias=True):
        self.bias_wt = np.zeros((no_of_curr_lay_node, 1)) if bias else None
        self.weight = np.random.rand(no_of_prev_lay_node, no_of_curr_lay_node)       
        self.no_of_prev_lay_node = no_of_prev_lay_node
        self.no_of_curr_lay_node = no_of_curr_lay_node
        self.last_lay = last_lay
        
        self.inp = None
        self.out = None
        self.delta_wt = None
        self.delta_in = None
        self.delta_b = None
              
    def lay_feed_forward(self, input):
        self.inp = input
        if self.bias_wt is not None:
            op_before_act = np.dot(self.weight.T, input)
        op_before_act = np.dot(self.weight.T, input)+self.bias_wt
        self.op_before_act = op_before_act
        self.out = op_before_act
        op_after_act = 1/(1+np.exp(-op_before_act))
        self.op_after_act = op_after_act
        return op_before_act
    
    def wt_grad(self, next_lay_wt = None, next_lay_grad = None, actual_output = None):
        if self.last_lay == False :
            #print("entered false")
            self.delta_wt = (self.op_after_act*(1-self.op_after_act))*np.dot(next_lay_wt, next_lay_grad)
        elif self.last_lay == True:
            #print("entered true")
            self.delta_wt = (self.op_after_act*(1-self.op_after_act))*(self.op_after_act-actual_output.T)
        self.delta_in = np.dot(self.inp, self.delta_wt.T)
        if self.bias_wt is not None:
            self.delta_b = np.sum(self.delta_wt, axis=-1, keepdims=True)/self.delta_wt.shape[-1]     
        

In [3]:
class mul_lay_prcp():
    def __init__(self, no_of_layers, layer_nodes):
        self.no_of_layers = no_of_layers
        self.layer_nodes = layer_nodes
        self.mlp_layers = []
        self.mlp_lay_out = []
        self.loss = []
        self.mlp_in = []
        self.mlp_out = []
        for lay_id in range(1,self.no_of_layers):
            if ((lay_id < self.no_of_layers-1) and (lay_id > 0)):
                self.mlp_layers.append(mul_lay_prcp_layer(self.layer_nodes[lay_id-1],self.layer_nodes[lay_id]))
            elif (lay_id == self.no_of_layers-1):
                self.mlp_layers.append(mul_lay_prcp_layer(self.layer_nodes[lay_id-1],self.layer_nodes[lay_id],last_lay = True))   
    
    def mlp_grad(self):
        for lay_id in reversed(range(len(self.mlp_layers))):
            #print(lay_id)
            if lay_id == len(self.mlp_layers)-1 and self.mlp_layers[lay_id].last_lay == True:
                self.mlp_layers[lay_id].wt_grad(actual_output = self.Y)
            else:
                self.mlp_layers[lay_id].wt_grad(next_lay_wt = self.mlp_layers[lay_id+1].weight, next_lay_grad = self.mlp_layers[lay_id+1].delta_wt)
                
    def train_mlp(self, X, Y, learn_rate, epochs):
        X = X.T
        Y = Y
        self.learn_rate = learn_rate
        count = int(epochs*0.1)
        for epoch_num in range(epochs):
            for x in range(X.shape[-1]):
                self.X = X[:,x].reshape(-1,1)
                self.Y = Y[x,:].reshape(1,-1)
                #out = self.mlp_feed_forward(self.X)
                mlp_f_out_1 = self.X
                for mlp_layer_1 in self.mlp_layers:
                    mlp_f_out_1 = 1/(1+np.exp(-(mlp_layer_1.lay_feed_forward(mlp_f_out_1))))
                    self.mlp_out.append(mlp_f_out_1)
                rm_sq_loss = self.root_mean_sqr_loss(mlp_f_out_1,self.Y)
                self.loss.append(rm_sq_loss)
                self.mlp_grad()
                self.SGD()
            #Z = self.mlp_feed_forward(X)
            #print(Z.T.shape. Y.shape)
            mlp_f_out_2 = X
            for mlp_layer_2 in self.mlp_layers:
                    mlp_f_out_2 = 1/(1+np.exp(-(mlp_layer_2.lay_feed_forward(mlp_f_out_2))))
            #Z = mlp_f_out_2
            accuracy = np.sum((mlp_f_out_2.T > 0.5) == Y)*100./Y.shape[0]
            if epoch_num%count==0:
                print("At epoch",epoch_num)
                print("Loss is ",self.loss[-1])
                print("Accuracy is",accuracy)
    
    def SGD(self):
        for mlp_layer in self.mlp_layers:
            #if mlp_layer.delta_in is not None:
            mlp_layer.weight = mlp_layer.weight-(self.learn_rate*mlp_layer.delta_in)
            if mlp_layer.delta_b is not None:
                mlp_layer.bias_wt = mlp_layer.bias_wt-(self.learn_rate*mlp_layer.delta_b)
            
    def test_mlp(self,X):
        X = X.T
        #out = self.mlp_feed_forward(X)
        out = X
        for mlp_layer_out in self.mlp_layers:
                    out = 1/(1+np.exp(-(mlp_layer_out.lay_feed_forward(out))))
        predictions = (out > 0.5).astype(np.int)
        return predictions
                
                
    @staticmethod
    def root_mean_sqr_loss(out,actual_output):
        out = out.T
        rm_sq_loss = (1/2)*np.sum(np.square(out - actual_output))
        return rm_sq_loss
                

In [4]:
mul_lyr_pcp = mul_lay_prcp(no_of_layers = 3, layer_nodes = [2,4,1])

In [5]:
X = np.array([
    [0,0],
    [1,0],
    [0,1],
    [1,1]
])
Y = np.array([
    [0],
    [1],
    [1],
    [0]
])

In [6]:
print("\nTraining phase of multi layer perceptron\n")
mul_lyr_pcp.train_mlp(X, Y, 1, 1500)
print("\nTesting phase of multi layer perceptron")
mul_lyr_pcp.test_mlp(X)


Training phase of multi layer perceptron

At epoch 0
Loss is  0.3039439048158523
Accuracy is 50.0
At epoch 150
Loss is  0.14884788883854203
Accuracy is 25.0
At epoch 300
Loss is  0.14311461294076042
Accuracy is 50.0
At epoch 450
Loss is  0.1402417279996087
Accuracy is 50.0
At epoch 600
Loss is  0.13515398489163846
Accuracy is 50.0
At epoch 750
Loss is  0.09359188417524367
Accuracy is 75.0
At epoch 900
Loss is  0.08315453331391819
Accuracy is 75.0
At epoch 1050
Loss is  0.008035527088264067
Accuracy is 100.0
At epoch 1200
Loss is  0.002408388560238789
Accuracy is 100.0
At epoch 1350
Loss is  0.0013389476011020618
Accuracy is 100.0

Testing phase of multi layer perceptron


array([[0, 1, 1, 0]])